In [1]:
import torch

In [2]:
t = torch.rand((2, 10)).cuda()
print(t)

tensor([[0.4162, 0.4802, 0.4890, 0.7483, 0.3273, 0.6707, 0.8435, 0.0545, 0.3783,
         0.8271],
        [0.0770, 0.7384, 0.1067, 0.8836, 0.0694, 0.4936, 0.3413, 0.2577, 0.5320,
         0.6702]], device='cuda:0')
